In [1]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
import random

[nltk_data] Downloading package brown to
[nltk_data]     /Users/shailynnxie/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import time


Redistribution des neuf catégories en trois grandes catégories:

In [2]:
themes = {}
themes["news"] = ['editorial','reviews','news']
themes["books"] = ["adventure","mystery","fiction","romance"]
themes["sciences"] = ["learned","government"]

Les statitstiques:

In [3]:
nb=0
label=[]
texte=[]
for category in themes.keys():
    print(category,":")
    print(len(brown.fileids(categories=themes[category])),"documents")
    print(len(brown.words(categories=themes[category])),"mots")

    nb+=len(brown.fileids(categories=themes[category]))
    for cat in themes[category]:
        for file in brown.fileids(categories=cat):
            label.append(category)
            texte.append(' '.join([i for i in brown.words(file)]))
print("Textes totals: ",nb)


news :
88 documents
202862 mots
books :
111 documents
265021 mots
sciences :
110 documents
252005 mots
Textes totals:  309


In [16]:
print("longueur de verteur : ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False)
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(texte).getnnz())


longueur de verteur : 
ngram_range (1, 1) :
237879
ngram_range (1, 2) :
758404
ngram_range (1, 3) :
1347775


In [14]:
print("longueur de verteur sans stopwords: ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False,stop_words='english')
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(texte).getnnz())

longueur de verteur sans stopwords: 
ngram_range (1, 1) :
197294
ngram_range (1, 2) :
504898
ngram_range (1, 3) :
827474


In [23]:
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPRegressor

liste_classifieurs= [
    ["Perceptron", Perceptron(eta0=0.1, random_state=0)],
    ["MultinomialNB", MultinomialNB()],
    ["Logistic Regression", LogisticRegression()],
    ["linear_svc", LinearSVC()],
    ["Random Forest",RandomForestClassifier(n_estimators=250,max_depth=4, random_state=0)],
    ["Decision Tree", DecisionTreeClassifier(criterion='gini',max_depth=4,splitter='random',min_samples_split=5)],
    ["Perceptron multicouche",MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(30,20), random_state=1)]
    
]
## Pour éviter les warnings:
import warnings 
warnings.filterwarnings("ignore")

In [27]:
Resultat=[]

In [25]:
import json
import os
chemin_expes = "brown_nltk_original.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 0


In [26]:
en_minuscules,enlever_stopwords  = False, False
from sklearn.metrics import classification_report 

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False)
    X = V.fit_transform(texte)
    y = label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(label)
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(rep ort)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.9355
('Temps_execution(secondes)', 0.04068112373352051)
  MultinomialNB classifier : 0.7204
('Temps_execution(secondes)', 0.005873918533325195)
  Logistic Regression classifier : 0.9570
('Temps_execution(secondes)', 0.45375680923461914)
  linear_svc classifier : 0.9570
('Temps_execution(secondes)', 0.028682947158813477)
  Random Forest classifier : 0.8925
('Temps_execution(secondes)', 0.3713672161102295)
  Decision Tree classifier : 0.6667
('Temps_execution(secondes)', 0.04468798637390137)
  Perceptron multicouche classifier : 0.8602
('Temps_execution(secondes)', 5.28367018699646)
---------------
Ngram_range : (1, 2)
  Perceptron classifier : 0.9355
('Temps_execution(secondes)', 0.03842496871948242)
  MultinomialNB classifier : 0.6882
('Temps_execution(secondes)', 0.02271103858947754)
  Logistic Regression classifier : 0.8495
('Temps_execution(secondes)', 2.3470940589904785)
  linear_svc classifier : 0.9462
('Temps_execut

In [28]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.956989247311828, "['linear_svc', 1, 1, False, False]"]
[0.956989247311828, "['Logistic Regression', 1, 1, False, False]"]
[0.946236559139785, "['linear_svc', 1, 2, False, False]"]
[0.9354838709677419, "['linear_svc', 1, 3, False, False]"]
[0.9354838709677419, "['Perceptron', 1, 2, False, False]"]
[0.9354838709677419, "['Perceptron', 1, 1, False, False]"]
[0.9139784946236559, "['Perceptron', 1, 3, False, False]"]
[0.9032258064516129, "['Perceptron multicouche', 1, 3, False, False]"]
[0.8924731182795699, "['Random Forest', 1, 1, False, False]"]
[0.8602150537634409, "['Perceptron multicouche', 1, 1, False, False]"]
[0.8494623655913979, "['Random Forest', 1, 3, False, False]"]
[0.8494623655913979, "['Logistic Regression', 1, 2, False, False]"]
[0.8172043010752689, "['Random Forest', 1, 2, False, False]"]
[0.8064516129032258, "['Perceptron multicouche', 1, 2, False, False]"]
[0.8064516129032258, "['Logistic Regression', 1, 3, False, False]"]
[0.7741935483870968, "['Decision Tree', 1, 3, 

In [43]:
en_minuscules,enlever_stopwords  = False, False
from sklearn.metrics import classification_report 

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False)
    X = V.fit_transform(texte)
    y = label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        clf = algo.fit(X_train, y_train)
        score = clf.score(X_train, y_train)
       
        print(nom,score)

---------------
Ngram_range : (1, 1)
Perceptron 1.0
MultinomialNB 0.9027777777777778
Logistic Regression 0.9953703703703703
linear_svc 1.0
Random Forest 0.9907407407407407
Decision Tree 0.9120370370370371
Perceptron multicouche 1.0
---------------
Ngram_range : (1, 2)
Perceptron 1.0
MultinomialNB 0.9675925925925926
Logistic Regression 1.0
linear_svc 1.0
Random Forest 0.9768518518518519
Decision Tree 0.9166666666666666
Perceptron multicouche 1.0
---------------
Ngram_range : (1, 3)
Perceptron 1.0
MultinomialNB 0.9814814814814815
Logistic Regression 1.0
linear_svc 1.0
Random Forest 0.9768518518518519
Decision Tree 0.9305555555555556


KeyboardInterrupt: 

In [30]:
chemin_expes = "brown_nltk_lower.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 0


In [31]:
en_minuscules,enlever_stopwords  = True, False

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = True)
    X = V.fit_transform(texte)
    y = label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(label)
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.9140
('Temps_execution(secondes)', 0.014909029006958008)
  MultinomialNB classifier : 0.7312
('Temps_execution(secondes)', 0.008095026016235352)
  Logistic Regression classifier : 0.9570
('Temps_execution(secondes)', 0.3043060302734375)
  linear_svc classifier : 0.9570
('Temps_execution(secondes)', 0.024254798889160156)
  Random Forest classifier : 0.9247
('Temps_execution(secondes)', 0.35959911346435547)
  Decision Tree classifier : 0.8172
('Temps_execution(secondes)', 0.04573512077331543)
  Perceptron multicouche classifier : 0.8925
('Temps_execution(secondes)', 6.56820821762085)
---------------
Ngram_range : (1, 2)
  Perceptron classifier : 0.9570
('Temps_execution(secondes)', 0.0323491096496582)
  MultinomialNB classifier : 0.6882
('Temps_execution(secondes)', 0.02165818214416504)
  Logistic Regression classifier : 0.8817
('Temps_execution(secondes)', 2.686736822128296)
  linear_svc classifier : 0.9462
('Temps_executi

In [32]:
json_str = json.dumps(dic_expes, indent=4) # 缩进4字符
with open(chemin_expes, 'a') as json_file:
	json_file.write(json_str)

In [33]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.956989247311828, "['linear_svc', 1, 1, False, True]"]
[0.956989247311828, "['Perceptron', 1, 2, False, True]"]
[0.956989247311828, "['Logistic Regression', 1, 1, False, True]"]
[0.946236559139785, "['linear_svc', 1, 2, False, True]"]
[0.9354838709677419, "['linear_svc', 1, 3, False, True]"]
[0.9247311827956989, "['Random Forest', 1, 1, False, True]"]
[0.9139784946236559, "['Perceptron', 1, 3, False, True]"]
[0.9139784946236559, "['Perceptron', 1, 1, False, True]"]
[0.9139784946236559, "['Perceptron multicouche', 1, 3, False, True]"]
[0.8924731182795699, "['Perceptron multicouche', 1, 1, False, True]"]
[0.8817204301075269, "['Logistic Regression', 1, 2, False, True]"]
[0.8602150537634409, "['Random Forest', 1, 3, False, True]"]
[0.8602150537634409, "['Perceptron multicouche', 1, 2, False, True]"]
[0.8387096774193549, "['Random Forest', 1, 2, False, True]"]
[0.8172043010752689, "['Logistic Regression', 1, 3, False, True]"]
[0.8172043010752689, "['Decision Tree', 1, 1, False, True]"]
[

In [34]:
chemin_expes = "brown_nltk_sw.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 0


In [35]:
en_minuscules,enlever_stopwords  = False, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False,stop_words='english')
    X = V.fit_transform(texte)
    y = label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(label)
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.9247
('Temps_execution(secondes)', 0.011800050735473633)
  MultinomialNB classifier : 0.9032
('Temps_execution(secondes)', 0.006679058074951172)
  Logistic Regression classifier : 0.9140
('Temps_execution(secondes)', 0.2810781002044678)
  linear_svc classifier : 0.9677
('Temps_execution(secondes)', 0.017364978790283203)
  Random Forest classifier : 0.8602
('Temps_execution(secondes)', 0.31847333908081055)
  Decision Tree classifier : 0.7742
('Temps_execution(secondes)', 0.04090714454650879)
  Perceptron multicouche classifier : 0.9247
('Temps_execution(secondes)', 2.6827070713043213)
---------------
Ngram_range : (1, 2)
  Perceptron classifier : 0.9140
('Temps_execution(secondes)', 0.025985240936279297)
  MultinomialNB classifier : 0.8172
('Temps_execution(secondes)', 0.028022050857543945)
  Logistic Regression classifier : 0.8710
('Temps_execution(secondes)', 2.4170849323272705)
  linear_svc classifier : 0.9247
('Temps_e

In [36]:
json_str = json.dumps(dic_expes, indent=4) # 缩进4字符
with open(chemin_expes, 'a') as json_file:
	json_file.write(json_str)

In [37]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.967741935483871, "['linear_svc', 1, 1, True, False]"]
[0.9247311827956989, "['linear_svc', 1, 2, True, False]"]
[0.9247311827956989, "['Perceptron', 1, 1, True, False]"]
[0.9247311827956989, "['Perceptron multicouche', 1, 1, True, False]"]
[0.9139784946236559, "['Perceptron', 1, 3, True, False]"]
[0.9139784946236559, "['Perceptron', 1, 2, True, False]"]
[0.9139784946236559, "['Logistic Regression', 1, 1, True, False]"]
[0.9032258064516129, "['MultinomialNB', 1, 1, True, False]"]
[0.8924731182795699, "['linear_svc', 1, 3, True, False]"]
[0.8709677419354839, "['Logistic Regression', 1, 2, True, False]"]
[0.8602150537634409, "['Random Forest', 1, 1, True, False]"]
[0.8494623655913979, "['Random Forest', 1, 2, True, False]"]
[0.8172043010752689, "['MultinomialNB', 1, 2, True, False]"]
[0.8064516129032258, "['Random Forest', 1, 3, True, False]"]
[0.7956989247311828, "['Perceptron multicouche', 1, 3, True, False]"]
[0.7956989247311828, "['Logistic Regression', 1, 3, True, False]"]
[0.7741

In [38]:
chemin_expes = "brown_nltk_sw_lower.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 0


In [39]:
en_minuscules,enlever_stopwords  = True, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = True,stop_words='english')
    X = V.fit_transform(texte)
    y = label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(label)
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.9355
('Temps_execution(secondes)', 0.008861064910888672)
  MultinomialNB classifier : 0.9032
('Temps_execution(secondes)', 0.0054950714111328125)
  Logistic Regression classifier : 0.9032
('Temps_execution(secondes)', 0.25968098640441895)
  linear_svc classifier : 0.9462
('Temps_execution(secondes)', 0.016668081283569336)
  Random Forest classifier : 0.8925
('Temps_execution(secondes)', 0.3201448917388916)
  Decision Tree classifier : 0.7742
('Temps_execution(secondes)', 0.041977882385253906)
  Perceptron multicouche classifier : 0.9032
('Temps_execution(secondes)', 2.3408148288726807)
---------------
Ngram_range : (1, 2)
  Perceptron classifier : 0.9355
('Temps_execution(secondes)', 0.025454998016357422)
  MultinomialNB classifier : 0.8495
('Temps_execution(secondes)', 0.01746511459350586)
  Logistic Regression classifier : 0.8602
('Temps_execution(secondes)', 2.2236077785491943)
  linear_svc classifier : 0.9247
('Temps_

In [40]:
json_str = json.dumps(dic_expes, indent=4) # 缩进4字符
with open(chemin_expes, 'a') as json_file:
	json_file.write(json_str)

In [41]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.946236559139785, "['linear_svc', 1, 1, True, True]"]
[0.9354838709677419, "['Perceptron', 1, 2, True, True]"]
[0.9354838709677419, "['Perceptron', 1, 1, True, True]"]
[0.9247311827956989, "['linear_svc', 1, 2, True, True]"]
[0.9032258064516129, "['linear_svc', 1, 3, True, True]"]
[0.9032258064516129, "['Perceptron', 1, 3, True, True]"]
[0.9032258064516129, "['Perceptron multicouche', 1, 1, True, True]"]
[0.9032258064516129, "['MultinomialNB', 1, 1, True, True]"]
[0.9032258064516129, "['Logistic Regression', 1, 1, True, True]"]
[0.8924731182795699, "['Random Forest', 1, 1, True, True]"]
[0.8602150537634409, "['Logistic Regression', 1, 2, True, True]"]
[0.8494623655913979, "['Random Forest', 1, 2, True, True]"]
[0.8494623655913979, "['MultinomialNB', 1, 2, True, True]"]
[0.7849462365591398, "['Perceptron multicouche', 1, 3, True, True]"]
[0.7849462365591398, "['Logistic Regression', 1, 3, True, True]"]
[0.7741935483870968, "['Random Forest', 1, 3, True, True]"]
[0.7741935483870968, "[

Résultat final:

In [42]:
for res in sorted(Resultat,reverse=True):
    print(res)

[0.967741935483871, "['linear_svc', 1, 1, True, False]"]
[0.956989247311828, "['linear_svc', 1, 1, False, True]"]
[0.956989247311828, "['linear_svc', 1, 1, False, False]"]
[0.956989247311828, "['Perceptron', 1, 2, False, True]"]
[0.956989247311828, "['Logistic Regression', 1, 1, False, True]"]
[0.956989247311828, "['Logistic Regression', 1, 1, False, False]"]
[0.946236559139785, "['linear_svc', 1, 2, False, True]"]
[0.946236559139785, "['linear_svc', 1, 2, False, False]"]
[0.946236559139785, "['linear_svc', 1, 1, True, True]"]
[0.9354838709677419, "['linear_svc', 1, 3, False, True]"]
[0.9354838709677419, "['linear_svc', 1, 3, False, False]"]
[0.9354838709677419, "['Perceptron', 1, 2, True, True]"]
[0.9354838709677419, "['Perceptron', 1, 2, False, False]"]
[0.9354838709677419, "['Perceptron', 1, 1, True, True]"]
[0.9354838709677419, "['Perceptron', 1, 1, False, False]"]
[0.9247311827956989, "['linear_svc', 1, 2, True, True]"]
[0.9247311827956989, "['linear_svc', 1, 2, True, False]"]
[0.